In [12]:
import torch
import torchvision
from torchvision.datasets import CIFAR10

In [14]:
from torchvision import transforms as T

# tfs = T.Compose( [T.Resize((28, 28)), T.Grayscale(num_output_channels = 3), T.ToTensor()] ) # MNIST
tfs = T.Compose( [T.Resize((32, 32)), T.ToTensor()] )

ds = CIFAR10(root = "datasets", train = True, download = True, transform = tfs)
ts_ds = CIFAR10(root = "datasets", train = False, download = True, transform = tfs)
classes = ds.classes
print(classes)
n_cls = len(classes)
n_cls

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


10

In [ ]:
from torch.utils.data import random_split, DataLoader

total_len = len(ds)
tr_len = int(total_len * 0.9)
vl_len = total_len - tr_len
tr_len, vl_len

(45000, 5000)

In [16]:
tr_ds, vl_ds = random_split(dataset = ds, lengths=[tr_len, vl_len])
print(len(tr_ds),len(vl_ds))

45000 5000


In [17]:
bs = 128
tr_dl = DataLoader(dataset = tr_ds, batch_size = bs, shuffle = True, num_workers = 8)
vl_dl = DataLoader(dataset = vl_ds, batch_size = bs, shuffle = False, num_workers = 8)
ts_dl = DataLoader(dataset = ts_ds, batch_size = 1,  shuffle = False, num_workers = 8)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## AI Model with Linear Layers

In [36]:
inp = torch.rand(1, 3, 32, 32) # 4D -> (bs, im_chs, im_h, im_w)
# 2 ni darajalari; 8 -> 16 -> 32
# stride -> filterning qadami
# conv = Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1, stride = 1) # Linear in_features, out_features; out_channels == filter soniga == kernellar soniga
# formula: [(32 + 2 * 1 - 3) / 1] + 1 = 32
# conv = Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 0, stride = 1) # Linear in_features, out_features; out_channels == filter soniga == kernellar soniga
# formula: [(32 + 0 - 3) / 1] + 1 = 29 + 1 = 30
conv = Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1, stride = 2) # Linear in_features, out_features; out_channels == filter soniga == kernellar soniga
# formula: [(32 + 2 - 3) / 2] + 1 = 15 + 1 = 16
mp = MaxPool2d(kernel_size = 2, stride = 2) # backpropda qatnashmaydi: trainable parametrlari yo'q
mp(inp).shape

# conv(inp).shape


torch.Size([1, 3, 16, 16])

In [43]:
from torch.nn import *

class CustomModel(Module):

  def __init__(self, in_chs, out_chs, ks, p, s, in_fs, out_fs, n_cls):
    super().__init__()

    self.conv_1 = Conv2d(in_channels = in_chs, out_channels = out_chs, kernel_size = ks, padding = p, stride = s)
    self.conv_2 = Conv2d(in_channels = out_chs, out_channels = out_chs * 2, kernel_size = ks, padding = p, stride = s)
    # self.conv_3 = Conv2d(in_channels = out_chs, out_channels = out_chs * 2, kernel_size = ks, padding = p, stride = s)

    self.lin_layer = Linear(in_features = in_fs, out_features = out_fs)
    self.classifier = Linear(in_features = out_fs, out_features = n_cls)
    self.act = ReLU() # prelu, leaky relu, elu,...
    self.mp = MaxPool2d(kernel_size = 2, stride = 2)

  def forward(self, inp):

    out = self.conv_1(inp)        # (bs, out_chs, im_h, im_w)
    out = self.mp(self.act(out))  # (bs, out_chs, im_h // 2, im_w // 2)
    out = self.conv_2(out)        # (bs, out_chs * 2, im_h // 2, im_w // 2)
    out = self.mp(self.act(out))  # (bs, out_chs * 2, im_h // 4, im_w // 4)

    bs = inp.shape[0]
    out = out.view(bs, -1)

    out = self.lin_layer(out)
    out = self.act(out)
    out = self.classifier(out)

    return out

inp = torch.rand(1,3,32,32)
out_fs = 256
m = CustomModel(in_chs = inp.shape[1], out_chs = 8, ks = 3, p = 1, s = 1, in_fs = 1024, out_fs = out_fs, n_cls = n_cls)
xom_bashorat = m(inp)
xom_bashorat.shape

torch.Size([1, 10])

Training loop

In [45]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model = m)

266362

In [44]:
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(m.parameters(), lr = 3e-4)

for epoch in range(epochs):

  # if epoch == 1: break
  tr_epoch_acc = 0
  tr_epoch_loss = 0
  vl_epoch_acc = 0
  vl_epoch_loss = 0

  for idx, batch in enumerate(tr_dl):

    rasmlar, javoblar = batch

    xom_bashoratlar = m(rasmlar)
    bashoratlar = torch.argmax(xom_bashoratlar, dim = 1)

    loss = loss_fn(xom_bashoratlar, javoblar) #
    tr_epoch_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    batch_acc = (bashoratlar == javoblar).sum().item()
    tr_epoch_acc += batch_acc

  print(f"{epoch + 1}-epochdagi train loss -> {tr_epoch_loss / len(tr_dl):.3f}")
  print(f"{epoch + 1}-epochdagi train acc  -> {tr_epoch_acc / len(tr_dl.dataset):.3f}")

  with torch.no_grad():
    for idx, batch in enumerate(vl_dl):
      rasmlar, javoblar = batch
      xom_bashoratlar = m(rasmlar)
      bashoratlar = torch.argmax(xom_bashoratlar, dim = 1)
      loss = loss_fn(xom_bashoratlar, javoblar)
      vl_epoch_loss += loss.item()

      batch_acc = (bashoratlar == javoblar).sum().item()
      vl_epoch_acc += batch_acc

  print(f"{epoch + 1}-epochdagi validation loss -> {vl_epoch_loss / len(vl_dl):.3f}")
  print(f"{epoch + 1}-epochdagi validation acc -> {vl_epoch_acc / len(vl_dl.dataset):.3f}")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


1-epochdagi train loss -> 1.922
1-epochdagi train acc  -> 0.313
1-epochdagi validation loss -> 1.688
1-epochdagi validation acc -> 0.399
2-epochdagi train loss -> 1.627
2-epochdagi train acc  -> 0.418
2-epochdagi validation loss -> 1.593
2-epochdagi validation acc -> 0.423
3-epochdagi train loss -> 1.528
3-epochdagi train acc  -> 0.453
3-epochdagi validation loss -> 1.490
3-epochdagi validation acc -> 0.468
4-epochdagi train loss -> 1.458
4-epochdagi train acc  -> 0.479
4-epochdagi validation loss -> 1.429
4-epochdagi validation acc -> 0.488
5-epochdagi train loss -> 1.396
5-epochdagi train acc  -> 0.504
5-epochdagi validation loss -> 1.371
5-epochdagi validation acc -> 0.505
6-epochdagi train loss -> 1.350
6-epochdagi train acc  -> 0.519
6-epochdagi validation loss -> 1.330
6-epochdagi validation acc -> 0.524
7-epochdagi train loss -> 1.301
7-epochdagi train acc  -> 0.541
7-epochdagi validation loss -> 1.321
7-epochdagi validation acc -> 0.526
8-epochdagi train loss -> 1.266
8-epochda